In [2]:
import pandas as pd
import altair as alt
import numpy as np
import sqlite3

# Create DataBase and Add Table

In [14]:
conn = sqlite3.connect('Netflix.db')

cur = conn.cursor()

# cur.execute("""CREATE TABLE NFLX (
#     date string,
#     low float,
#     open float,
#     volume integer,
#     high float,
#     close float
# )""")

# Fill Table

In [4]:
path = 'C:/Users/Blake Dennett/Downloads/Spring2023/appliedProgramming/Data/stock_market_data/sp500/csv/j-p/nflx.csv'
df = pd.read_csv(path)
# iterate through each row of dataframe
for r in df.iterrows():
    row = r[1].tolist()
    to_insert = tuple(row[0:6])
    # add row into db
    # cur.execute(f"INSERT INTO NFLX VALUES {to_insert}")

conn.commit()

five_rows = pd.read_sql_query( 
    'SELECT * FROM NFLX LIMIT 5',
    conn)

five_rows

    

date       low      open     volume      high     close difference
0  23-05-2002  1.145714  1.156429  104790000  1.242857  1.196429       None
1  24-05-2002  1.197143  1.214286   11104800  1.225000  1.210000       None
2  28-05-2002  1.157143  1.213571    6609400  1.232143  1.157143       None
3  29-05-2002  1.085714  1.164286    6757800  1.164286  1.103571       None
4  30-05-2002  1.071429  1.107857   10154200  1.107857  1.071429       None

# Deleting Duplicate Data

In [9]:
# cur.execute("DELETE FROM nflx WHERE date NOT IN (SELECT MIN(date) FROM nflx GROUP BY date);")

# Filtering Data

In [5]:
cur.execute("SELECT * FROM NFLX WHERE date = '23-05-2002'")
cur.fetchall()

[('23-05-2002',
  1.1457140445709229,
  1.1564290523529053,
  104790000,
  1.2428569793701172,
  1.1964290142059326,
  None)]

# Creating New Data Table

In [ ]:
# cur.execute("""CREATE TABLE NFLX_ANALYSIS (
#             date string,
#             difference float
# )""")

# Fill Date Column From One Table to Another

In [25]:
data_transfer = """INSERT INTO NFLX_ANALYSIS (date) 
SELECT date FROM NFLX"""
# cur.execute(data_transfer)

data_display = """SELECT * 
FROM NFLX_ANALYSIS 
LIMIT 5"""

print(pd.read_sql_query(data_display, conn))

         date       low      open     volume      high     close  difference
0  23-05-2002  1.145714  1.156429  104790000  1.242857  1.196429    0.097143
1  24-05-2002  1.197143  1.214286   11104800  1.225000  1.210000    0.027857
2  28-05-2002  1.157143  1.213571    6609400  1.232143  1.157143    0.075000
3  29-05-2002  1.085714  1.164286    6757800  1.164286  1.103571    0.078572
4  30-05-2002  1.071429  1.107857   10154200  1.107857  1.071429    0.036428


# Use Join to add difference column

In [29]:
# make a difference column
query = """ALTER TABLE NFLX
ADD COLUMN difference FLOAT;"""
# cur.execute(query)

# subtract the high from the low
query = """UPDATE NFLX
SET difference = high - low"""
# cur.execute(query)

# enter difference into NFLX_ANALYSIS
diff_transfer = """INSERT INTO NFLX_ANALYSIS (difference, date)
SELECT difference, date FROM NFLX"""
# cur.execute(diff_transfer)

# drop difference column from NFLX
query = """ALTER TABLE NFLX
DROP COLUMN difference;"""
# cur.execute(query)


# cur.execute("""DELETE FROM NFLX_ANALYSIS""")


print(pd.read_sql_query("""SELECT * FROM NFLX_ANALYSIS LIMIT 5""", conn))
print(pd.read_sql_query("""SELECT * FROM NFLX_ANALYSIS WHERE date IS NULL""", conn))
print(pd.read_sql_query("""SELECT * FROM NFLX WHERE date = '23-05-2002'""", conn))

conn.commit()



         date  difference
0  23-05-2002    0.097143
1  24-05-2002    0.027857
2  28-05-2002    0.075000
3  29-05-2002    0.078572
4  30-05-2002    0.036428
Empty DataFrame
Columns: [date, difference]
Index: []
         date       low      open     volume      high     close
0  23-05-2002  1.145714  1.156429  104790000  1.242857  1.196429


# Deleting Duplicate Rows

In [23]:
# keeps last instance of a duplicate
query = """DELETE FROM NFLX_ANALYSIS 
WHERE rowid NOT IN (SELECT max(rowid) FROM NFLX_ANALYSIS GROUP BY date)"""
cur.execute(query)

query = """DELETE FROM NFLX 
WHERE rowid NOT IN (SELECT max(rowid) FROM NFLX GROUP BY date)"""
cur.execute(query)

conn.commit()

# Close the Connection

In [ ]:
conn.close()